In this assignment, you'll be working with functions, generators, and decorators.

## Instructions

- Inside the solutions folder (`programming3-class/assignment_functions/solutions/`), create a folder with your name lowercase (for e.g., if your name is Anier, then your folder would be named `anier`).
- Copy the given notebook inside the corresponding folder (in my case, I would have to copy it to `programming3-class/assignment_functions/solutions/anier/`) and solve the tasks.
- Each of the tasks contains a line saying `raise NotImplementedError`. This is there in purpose. Once you've implemented a task, remove that line on that task.
- Once you're done, make a PR with your commits.

## Task 1

Write a function with documentation in a docstring (no need for a complicated documentation, just something very simple describing the function).
- The function should contain all types of arguments (positional, positional with defaults, arbitrary args, keyword args, arbitrary keyword args).

In [1]:
def my_function(positional_arg, default_arg=10, *args, **kwargs):
    """
    This function does nothing interesting.

    Args:
    positional_arg: A required positional argument.
    default_arg: An optional positional argument with a default value of 10.
    *args: Arbitrary positional arguments.
    **kwargs: Arbitrary keyword arguments.
    """
    print("positional_arg:", positional_arg)
    print("default_arg:", default_arg)
    print("*args:", args)
    print("**kwargs:", kwargs)

## Task 2

A number is prime if it has 2 divisors exactly.

The first few primes are: 2, 3, 5, 7, 11, ...



Make a function that checks if a number is prime or not.

In [2]:
from math import sqrt

def is_prime(n: int) -> bool:
    if n <= 1:
        return False
    if n % 2 == 0 or n % 3 == 0:
        return False
    limit = int(sqrt(n))
    for i in range(5, limit+1, 6):
        if n % i == 0 or n % (i+2) == 0:
            return False
    return True


## Task 3

Implement the following function according to the docstring provided.

Read https://docs.python.org/3.11/library/inspect.html in order to see how to get each requested information.

In [12]:
import inspect

def inspect_function(func):
    """
    Takes another function as an argument (but not built-in)
    and print the following data:
    the name of the analyzed function,
    the name of all the arguments it takes
    and their types (positional, keyword, etc.)
    """
    signature = inspect.signature(func)
    print("Function Name:", func.__name__)
    for param in signature.parameters.values():
        print("Parameter:", param.name, " - ", param.kind)

In [13]:
inspect_function(my_function)

Function Name: my_function
Parameter: positional_arg  -  POSITIONAL_OR_KEYWORD
Parameter: default_arg  -  POSITIONAL_OR_KEYWORD
Parameter: args  -  VAR_POSITIONAL
Parameter: kwargs  -  VAR_KEYWORD


## Task 4

The following function isn't working properly.
Fix it so that it prints the current datetime with a message.

In [17]:
from datetime import datetime
from time import sleep

# wrong function
def my_time_now(msg):
    print(msg, datetime.now())

In [23]:
# simple tests :)
my_time_now('The time is now: ')
sleep(1)
my_time_now('The time is now: ')
sleep(1)
my_time_now('The time is now: ')

The time is now:  2024-02-07 00:02:28.710020
The time is now:  2024-02-07 00:02:29.711521
The time is now:  2024-02-07 00:02:30.713106


## Task 5

Make a function that returns at most `max_count` values of a given generator.

In [24]:
def limit(input_generator, max_count):
    """
    Generator that returns not more than max_count values of the input_generator.
    """
    count = 0
    for value in input_generator:
        if count < max_count:
            yield value
            count += 1
        else:
            break

In [25]:
def my_generator():
    for i in range(1000):
        yield i

limited_values = list(limit(my_generator(), 5))
print(limited_values)

[0, 1, 2, 3, 4]


## Task 6

Write a generator for an infinite sequence of numbers from the Pascal's triangle. The sequence look like this: `1 1 1 1 2 1 1 3 3 1 1 4 6 4 1 1 5 10 10 5 1 1 6 15 20 15 6 1 1 7 21 35 35 21 7 1 1 8 28 56 70 56 28 8 1 1 9 36 84 126 126 84 36 9 1 ... '

Test it with a generator from the previous task

In [31]:
def pascals_triangle():
    row = [1]
    while True:
        for num in row:
            yield num
        row = [1] + [row[i] + row[i + 1] for i in range(len(row) - 1)] + [1]

list(limit(pascals_triangle(), 19))

[1, 1, 1, 1, 2, 1, 1, 3, 3, 1, 1, 4, 6, 4, 1, 1, 5, 10, 10]

## Task 7

Write a `merge_sorter` generator that merges sorted sequences of integers.

The generator takes an arbitrary number of arguments. The argument can be any iterable, including another generator. It is guaranteed that each argument is a sequence of integers, sorted in non-decreasing order.

In [32]:
def merge_sorter(*args):
    iterators = [iter(arg) for arg in args]
    current = [next(it) for it in iterators]
    while True:
        min_val, min_idx = min((val, idx) for idx, val in enumerate(current) if val is not None)
        yield min_val
        try:
            current[min_idx] = next(iterators[min_idx])
        except StopIteration:
            current[min_idx] = None
            if all(val is None for val in current):
                break

In [35]:
test_sequence_1 = [1, 3, 5, 7, 9]
test_sequence_2 = [2, 4, 6, 8, 10]
test_sequence_3 = [0, 2, 4, 6, 8]
merged_generator = merge_sorter(test_sequence_1, test_sequence_2, test_sequence_3)
print(list(merged_generator))

[0, 1, 2, 2, 3, 4, 4, 5, 6, 6, 7, 8, 8, 9, 10]


## Task 8

Write the decorator proﬁler, which, when calling a function, will store in its attributes (not to be confused with arguments) the time of its execution (in seconds, it can be fractional) and the number of recursive calls that occurred during execution. Name the attributes `last_time_taken` and `calls`. It is forbidden to use global variables. The decorator must behave in a decent manner, that is, it must not overwrite the function's documentation.

For tests write [Ackemann function](https://en.wikipedia.org/wiki/Ackermann_function).

In [67]:
import time
import functools

def profiler(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if not hasattr(wrapper, 'calls'):
            wrapper.calls = 0
        wrapper.calls += 1
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        wrapper.last_time_taken = end_time - start_time
        return result
    return wrapper

@profiler
def ackermann(m, n):
    if m == 0:
        return n + 1
    elif n == 0:
        return ackermann(m - 1, 1)
    else:
        return ackermann(m - 1, ackermann(m, n - 1))

In [70]:
result = ackermann(1, 1)

print(f"Time taken for the last call: {ackermann.last_time_taken} seconds")
print(f"Number of recursive calls: {ackermann.calls}")

Time taken for the last call: 9.298324584960938e-06 seconds
Number of recursive calls: 24
